In [1]:
# data_processing.py

import pandas as pd
import numpy as np
import io
from config import SENSOR_COLS

def load_csv(uploaded_file):
    """
    Carga un archivo CSV desde un objeto UploadedFile de Streamlit.
    """
    if uploaded_file is not None:
        uploaded_file.seek(0)  # Reset file pointer
        return pd.read_csv(uploaded_file, encoding='latin1')
    return pd.DataFrame() # Return empty DataFrame if no file

def extract_patient_data(df):
    """
    Extrae datos personales y de configuración del paciente desde un DataFrame.
    Asume que los nombres de columna son consistentes.
    """
    col_map = {col.lower().strip(): col for col in df.columns}

    datos = {
        "sexo": "No especificado",
        "edad": np.nan, # Usar NaN para valores numéricos ausentes
        "mano_medida": "No especificada",
        "dedo_medido": "No especificado",
        "Nombre": None,
        "Apellido": None,
        "Diagnostico": None,
        "Antecedente": None,
        "Medicacion": None,
        "Tipo": None,
        "ECP": None, "GPI": None, "NST": None, "Polaridad": None,
        "Duracion": np.nan, "Pulso": np.nan, "Corriente": np.nan, # Usar NaN para numéricos
        "Voltaje": np.nan, "Frecuencia": np.nan
    }

    if not df.empty:
        # Extraer datos personales y de configuración
        for key_orig, default_val in datos.items():
            key_l = key_orig.lower()
            if key_l in col_map and pd.notna(df.at[0, col_map[key_l]]):
                val = str(df.at[0, col_map[key_l]]).strip().replace(',', '.')
                try:
                    if key_orig in ["edad", "Duracion", "Pulso", "Corriente", "Voltaje", "Frecuencia"]:
                        datos[key_orig] = float(val)
                    else:
                        datos[key_orig] = val
                except ValueError:
                    datos[key_orig] = val # Keep as text if not convertible
    return datos

def clean_sensor_data(df):
    """
    Limpia y valida las columnas de datos del sensor.
    Convierte a numérico y elimina filas donde TODOS los datos del sensor son NaN.
    """
    df_cleaned = df.copy()

    # Convertir a numérico las columnas del sensor
    for col in SENSOR_COLS:
        if col in df_cleaned.columns:
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
        else:
            # Si la columna no existe, se crea con NaN y la advertencia se manejará en la UI de Streamlit
            df_cleaned[col] = np.nan

    # Eliminar filas donde TODOS los datos del sensor son NaN
    # Esto es crucial para no intentar procesar filas sin datos relevantes
    df_cleaned = df_cleaned.dropna(subset=SENSOR_COLS, how='all')
    df_cleaned = df_cleaned.reset_index(drop=True)
    return df_cleaned


ModuleNotFoundError: No module named 'config'